# Persistente Transpositionstabelle für Rote-Learning

In [ ]:
import os.path
css = ""
if os.path.isfile("style.html"):
    from IPython.core.display import HTML
    with open("style.html", "r") as file:
        css = file.read()
HTML(css)

Der Training Prozess des Rote-Learnings ist sehr zeitaufwendig, da viele Spiele gespielt werden müssen. Damit dieser Prozess nicht vor jedem Spiel ausgeführt werden muss, ist es sinnvoll die trainierte Transpositionstabelle persistent auf der Festplatte abspeichern zu können. Diese Implementierung ist äquivalent zur Implementierung in `nmm-cache.ipynb`, allerdings wurden hier nicht benötigte Funktionen entfernt: das Speichern von `alpha`, `beta` und `limit` sowie die `clean` Methode.

## CacheRoteLearning
Die Klasse `CacheRoteLearning` implementiert eine Transpositionstabelle für den Rote-Learning Algorithmus, die persistiert werden kann.
Der Konstruktor erhält den folgenden Parameter:
- `path` gibt an, aus welcher Datei der Cache geladen werden soll. Ist dieser nicht gesetzt wird ein neuer leerer Cache initiiert.

In [ ]:
class CacheRoteLearning:
    def __init__(self, path: str = None):
        self.cache = {}
        if path:
            self.load(path)

Für Entwicklungszwecke wird eine Stringdarstellung für die Klasse `CacheRoteLearning` implementiert. Hierzu wird durch die Funktion `__repr__` ein String zurückgegeben, der alle Parameter der Klasse beinhaltet.

In [ ]:
def __repr__(self: CacheRoteLearning) -> str:
    return f"CacheRoteLearning(size={len(self.cache)})"

CacheRoteLearning.__repr__ = __repr__
del __repr__

Damit die Zustände und die Werte direkt als Bytes gespeichert und wieder ausgelesen werden können, ist das Paket `struct` nötig. Dieses ermöglicht `float` Werte zu `bytes` zu konvertieren. Das Paket `tqdm` ermöglicht eine simple Fortschrittsanzeige.

In [ ]:
import struct
from tqdm.notebook import tqdm

Die Methode `convert_state_to_bytes` konvertiert einen Zustand in ein Byte-Array der Länge 7. Diese kann aus der normalen `Cache` Implementierung geladen werden.

In [ ]:
%run ./nmm-cache.ipynb

Die Methode `write` schreibt einen Zustand in die Transpositionstabelle. Dabei wird der Zustand auf den weißen Spieler normiert. Es werden folgende Argumente erwartet:
- `state` $\in States$;
- `player` $\in Player$;
- `value` $\in \mathopen[-1.0,1.0\mathclose]$.

In [ ]:
def write(self: CacheRoteLearning, state, player: str, value: float) -> None:
    key = convert_state_to_bytes(state, player)
    if player == 'b':
        value = -value
    value = struct.pack("d", value)
    self.cache[key] = value

CacheRoteLearning.write = write
del write

Die Methode `read` liest einen vorher gespeicherten Zustand aus der Transpositionstabelle aus. Falls der Zustand nicht vorhanden ist wird `None` zurückgegeben. Dabei wird die Normierung auf den weißen Spieler rückgängig gemacht. Folgende Argumente werden erwartet:
- `state` $\in States$;
- `player` $\in Player$;
- `limit` $\in \mathbb{N}_0$.

Zurückgegeben wird ein Tripel bestehend aus:
1. `value` $\in \mathopen[-1.0,1.0\mathclose]$;
2. `alpha` $\in \mathopen[-1.0,1.0\mathclose]$;
3. `beta` $\in \mathopen[-1.0,1.0\mathclose]$.

In [ ]:
def read(self: CacheRoteLearning, state, player: str) -> float:
    key = convert_state_to_bytes(state, player)
    result = self.cache.get(key)
    if not result:
        return None
    value = struct.unpack("d", result)[0]
    if player == 'b':
        value = -value
    return value

CacheRoteLearning.read = read
del read

Die Methode `save` persistiert den Cache auf dem Dateisystem. Dafür wird folgendes Argument erwartet:
- `path` beschreibt den Pfad zur zu schreibenden Datei im Dateisystem.

In [ ]:
def save(self: CacheRoteLearning, path: str):
    with open(path, "wb") as file:
        for key,value in tqdm(self.cache.items()):
            file.write(key)
            file.write(value)

CacheRoteLearning.save = save
del save

Die Methode `load` lädt einen zuvor persistierten Cache aus einer Datei, die sich auf dem Dateisystem des Computers befindet. Dafür wird folgendes Argument erwartet:
- `path` beschreibt den Pfad zur zu lesenden Datei im Dateisystem.

In [ ]:
def load(self: CacheRoteLearning, path: str):
    if not os.path.isfile(path):
        print(f'Failed to load cache from file {path}!')
        return
    with open(path, "rb") as file:
        while True:
            key = file.read(7)
            value = file.read(8)
            if not key or not value:
                break
            self.cache[key] = value
    print(f'Successfully loaded cache from file {path}!')
CacheRoteLearning.load = load
del load